# The probe phase
There are several ways of analyzing the topology and trajectory pairs, depending on the number of trajectory files per a topology file, the continuity of trjaectory files, organization of files in a directory, and the parallel or sequencial arrangement of the computation powerhorse.


### To-do-list

This notebook is a living documnent. It is an integration of all the past notebooks and scripts written for the *probe* phase in the *PolyPhys* package (or formerly-called *extraction* phase in the decryped *sumrule* package)

BThe list below allows to review the past notebooks and scripts and combine them into this notebook:

- [] Sum-rule: segments: bug test and run on **date:NODATE-YET**
- [] Sum-rule: segments: all test and run on **date:NODATE-YET**
- [] Sum-rule: wholes: bug test and run on **date:NODATE-YET**
- [] Sum-rule: wholes: all test and run on **date:NODATE-YET**
- [] Sum-rule: all_in_one: test and run on **date:NODATE-YET**
- [X] Trans-Foci: segments: bug test and run on **date:20220621**
- [] Trans-Foci: segments: all test and run on **date:NODATE-YET**
- [] Trans-Foci: wholes: bug test and run on **date:NODATE-YET**
- [] Trans-Foci: wholes: all test and run on **date:NODATE-YET**
- [] Trans-Foci: all_in_one: test and run on **date:NODATE-YET**

### Naming convention:

This is the pattern of file or directory names:

1. **whole** files: whole-group-property_[-measure][-stage][.ext]
2. **ensemble** files: ensemble-group-property_[-measure][-stage][.ext]
3. **ensemble_long** files: ensemble_long-group-property_[-measure][-stage][.ext]
4. **space** files: space-group-property_[-measure][-stage][.ext]
5. **all in one** files: space-group-**species**-**allInOne**-property-_[-measure][-stage][.ext]

[keyword] means that the keyword in the file name is option. [-measure] is a physical measurement such as the auto correlation function (AFC) done on the physical 'property_'.

### Settings for testing and running on a PC.

In [1]:
# settings for testing and running on a PC.
from glob import glob
from polyphys.manage import organizer
from polyphys.manage.parser import SumRule, TransFoci
from polyphys.probe import prober

# HPC Cluster: gnuparallel

## Separated *whole* simulation directories

On a cluster, *whole* simulations are organized into *whole* directories, where each *whole* directory contains all the files for a given *whole* simulation. The **gnuparallel** is used to parallalize the **probe** phase at the **shell** level. For this purpose, all the python modules and scripts are separatedly installed and run on each core. For instance, if 32 cores are available, then the files in 32 *whole* directories are simulatenously installed. However, each *whole* directory may contains multiple toplogy and trajectory pairs. Thus, there is parallelization at the level of *whole* directories, not at the levle of the *segment* or *whole* trajectories inside a *whole* directory. Inside each *whole* directory, a python **main_probe.py** script analyzes the trajectories in a sequencial way.

- trj and all *segments* on a cluster

For each *whole* directory, the following script is executed by means of *gnuparallel*. See these scripts: *probe-1.7-all_trj_segments.py* and *probe-1.7-bug_trj_segments*

### Sum-rule project

#### whole bug trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober


# analyzing bug files.
geometry = 'biaxial'
group = 'bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug(
        bug_topo,
        bug_trj,
        geometry,
        lineage,
        save_to = save_to
    )

#### segment bug trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRule


geometry = 'biaxial'
group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob('./N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob('./N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRule(
        bug_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug(
            bug_topo, bug_trj, geometry, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug(
            bug_topo,
            bug_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRule


geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./N*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRule(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all(
            all_topo, all_trj, geometry, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_all(
            all_topo,
            all_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

### Trans-Foci project

#### whole bug trjs
Each *bug topology* comes with only **one** *bug trajectory*.

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober

# analyzing bug files.
geometry = 'biaxial'
group='bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug(
        bug_topo, bug_trj, lineage, group, geometry, save_to=save_to
    )

#### segment all trjs

In [ ]:
geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFoci(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all(
            all_topo, all_trj, geometry, lineage, save_to=save_to
        )
    else:

        prober.trans_foci_all(
            all_topo,
            all_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

# PC Serial scheme 

There are 4 different types of directories from which only one type can be in a *space* directory.

There are separated **whole** directories in each of which there **all** and **bug** **whole** trajectories; or, there are again separated **whole** directories in each of which there are **all** and **bug** **segment** trjaectories. Below there are two groups of scrips for **serial** and **parallel** runnung schemes.


On a PC, the *whole* directories are located in a master *space-trjs* directory; however, one main python script probes all the *whole* directories in a parallel scheme via Dask. This is different from the *gnuparallel*-based approach in which each *whole* directory has its own copy of the required scripts and a main pytohn script is run to probe that direcotry individually.

### Sumrule project

#### whole bug trjs

In [ ]:
# analyzing bug files.
geometry = 'biaxial'
group = 'bug'
lineage = 'whole'
save_to = './'
macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
bug_pairs = glob(macbookpro_path + '/N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug(
        bug_topo,
        bug_trj,
        geometry,
        lineage,
        continuous=False,
        save_to = save_to
    )

#### segment bug trjs

In [ ]:
# 1 min and 45 s for ~30000 particles with one all trj
macbookpro_path = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
geometry = 'biaxial'
group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob(macbookpro_path + '/N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob(macbookpro_path + '/N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRule(
        bug_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug(
            bug_topo, bug_trj, geometry, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug(
            bug_topo,
            bug_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs

In [2]:
from typing import Optional
from typing import Callable, Optional, Tuple, Dict, Type
import MDAnalysis as mda
import numpy as np
import pandas as pd
import os
from polyphys.manage.parser import SumRule, TransFoci
from polyphys.manage.organizer import invalid_keyword
from polyphys.analyze import clusters
import warnings

In [ ]:
# 4 mins for ~4000 particles with one all trj
# macbookmini
parent = "/Users/amirhsi_mini/research_data/trjs"
# macbookpro
#parent = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
continuous = True
lineage = 'segment'
save_to = './'
all_trjs = glob(parent + '/N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
print(all_trjs)
all_topo = glob(parent + '/N*' + group + '*')
print(all_topo)
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs[:1]:
    trj_info = SumRule(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    trajectory = all_trj
    topology = all_topo
    if (lineage == 'segment') & (continuous is False):
        print(
            "lineage is "
            f"'{lineage}' "
            "and 'continuous' is "
            f"'{continuous}. "
            "Please ensure the "
            f"'{trajectory}' is NOT part of a sequence of trajectories.")
    print("Setting the name of analyze file...\n")
    sim_info = SumRule(
        trajectory,
        geometry,
        'all',
        lineage
    )
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    print("\n" + sim_name + " is analyzing...")
    # dict of bin edges:
    bin_edges = {
        'xEdge': {
            'bin_size':  0.1 * min(sim_info.dmon, sim_info.dcrowd),
            'lmin': -0.5 * sim_info.dcyl,
            'lmax': 0.5 * sim_info.dcyl
            },
        'yEdge': {
            'bin_size':  0.1 * min(sim_info.dmon, sim_info.dcrowd),
            'lmin': -0.5 * sim_info.dcyl,
            'lmax': 0.5 * sim_info.dcyl
            },
        'zEdge': {
            'bin_size':  0.5 * min(sim_info.dmon, sim_info.dcrowd),
            'lmin': -0.5 * sim_info.lcyl,
            'lmax': 0.5 * sim_info.lcyl
            }
        }
    # LJ time difference between two consecutive frames:
    time_unit = \
        sim_info.dmon * np.sqrt(sim_info.mmon * sim_info.eps_others)
    lj_nstep = sim_info.adump  # Sampling steps via dump command in Lammps
    lj_dt = sim_info.dt
    sim_real_dt = lj_nstep * lj_dt * time_unit
    cell = mda.Universe(
        topology,
        trajectory,
        topology_format='DATA',
        format='LAMMPSDUMP',
        lammps_coordinate_convention='unscaled',
        atom_style="id resid type x y z",
        dt=sim_real_dt
    )
    # slicing trajectory based the continuous condition
    if continuous:
        sliced_trj = cell.trajectory[0: -1]
    else:
        sliced_trj = cell.trajectory
    # selecting atom groups
    crds = cell.select_atoms('resid 0')  # crowders
    bug = cell.select_atoms('resid 1')  # the chain or monomers
    # bin edges and histograms in different directions:
    # x direction of the cylindrical coordinate system
    x_hist_crd_info = prober.fixedsize_bins(
        sim_name,
        'xEdgeCrd',
        bin_edges['xEdge']['bin_size'],
        bin_edges['xEdge']['lmin'],
        bin_edges['xEdge']['lmax'],
        bin_type='ordinary',
        save_edge=False,
        save_to=save_to
    )
    x_hist_mon_info = prober.fixedsize_bins(
        sim_name,
        'xEdgeMon',
        bin_edges['xEdge']['bin_size'],
        bin_edges['xEdge']['lmin'],
        bin_edges['xEdge']['lmax'],
        bin_type='ordinary',
        save_edge=False,
        save_to=save_to
    )
    # y direction of the cylindrical coordinate system
    y_hist_crd_info = prober.fixedsize_bins(
        sim_name,
        'yEdgeCrd',
        bin_edges['yEdge']['bin_size'],
        bin_edges['yEdge']['lmin'],
        bin_edges['yEdge']['lmax'],
        bin_type='ordinary',
        save_edge=False,
        save_to=save_to
    )
    y_hist_mon_info = prober.fixedsize_bins(
        sim_name,
        'yEdgeMon',
        bin_edges['yEdge']['bin_size'],
        bin_edges['yEdge']['lmin'],
        bin_edges['yEdge']['lmax'],
        bin_type='ordinary',
        save_edge=False,
        save_to=save_to
    )
    # z direction of the cylindrical coordinate system
    z_hist_crd_info = prober.fixedsize_bins(
        sim_name,
        'zEdgeCrd',
        bin_edges['zEdge']['bin_size'],
        bin_edges['zEdge']['lmin'],
        bin_edges['zEdge']['lmax'],
        bin_type='ordinary',
        save_edge=False,
        save_to=save_to
    )
    z_hist_mon_info = prober.fixedsize_bins(
        sim_name,
        'zEdgeMon',
        bin_edges['zEdge']['bin_size'],
        bin_edges['zEdge']['lmin'],
        bin_edges['zEdge']['lmax'],
        bin_type='ordinary',
        save_edge=False,
        save_to=save_to
    )
    # check if any of the histograms are empty or not.
    if any([
            x_hist_mon_info['collector'].any() != 0,
            x_hist_crd_info['collector'].any() != 0,
            x_hist_mon_info['collector_std'].any() != 0,
            x_hist_crd_info['collector_std'].any() != 0,
            y_hist_mon_info['collector'].any() != 0,
            y_hist_crd_info['collector'].any() != 0,
            y_hist_mon_info['collector_std'].any() != 0,
            y_hist_crd_info['collector_std'].any() != 0,
            z_hist_mon_info['collector'].any() != 0,
            z_hist_crd_info['collector'].any() != 0,
            z_hist_mon_info['collector_std'].any() != 0,
            z_hist_crd_info['collector_std'].any() != 0,
            ]):
        raise ValueError(
            "One of the histogram collectors is not empty!")
    # 3D hist
    hist_crd_info_3d = {
        'n_bins': (
            x_hist_crd_info['n_bins'],
            y_hist_crd_info['n_bins'],
            z_hist_crd_info['n_bins']
            ),
        'bin_edges': [
            x_hist_crd_info['bin_edges'],
            y_hist_crd_info['bin_edges'],
            z_hist_crd_info['bin_edges']
            ],
        'range':[
            list(x_hist_crd_info['range']),
            list(y_hist_crd_info['range']),
            list(z_hist_crd_info['range'])
        ] 
    }
    hist_crd_info_3d['collector'] = np.zeros(hist_crd_info_3d['n_bins'])
    hist_crd_info_3d['collector'] *= 0
    hist_crd_info_3d['collector_std'] = hist_crd_info_3d['collector'].copy()
    hist_mon_info_3d = {
        'n_bins': (
            x_hist_mon_info['n_bins'],
            y_hist_mon_info['n_bins'],
            z_hist_mon_info['n_bins']
            ),
        'bin_edges': [
            x_hist_mon_info['bin_edges'],
            y_hist_mon_info['bin_edges'],
            z_hist_mon_info['bin_edges']
            ],
        'range': [
            list(x_hist_mon_info['range']),
            list(y_hist_mon_info['range']),
            list(z_hist_mon_info['range'])
        ]
    }
    hist_mon_info_3d['collector'] = np.zeros(hist_mon_info_3d['n_bins'])
    hist_mon_info_3d['collector'] *= 0
    hist_mon_info_3d['collector_std'] = hist_mon_info_3d['collector'].copy()
    for _ in sliced_trj:
        # histogram in 3 dimesional space
        # crds
        frame_hist, _ = np.histogramdd(
            crds.positions,
            bins=(x_hist_crd_info['bin_edges'],
                  y_hist_crd_info['bin_edges'],
                  z_hist_crd_info['bin_edges']),
            range=(x_hist_crd_info['range'],
                   y_hist_crd_info['range'],
                   z_hist_crd_info['range'])
        )
        hist_crd_info_3d['collector'] += frame_hist
        hist_crd_info_3d['collector_std'] += np.square(frame_hist)
        # bug
        frame_hist, _ = np.histogramdd(
            bug.positions,
            bins=(x_hist_mon_info['bin_edges'],
                  y_hist_mon_info['bin_edges'],
                  z_hist_mon_info['bin_edges']),
            range=(x_hist_mon_info['range'],
                   y_hist_mon_info['range'],
                   z_hist_mon_info['range'])
        )
        hist_mon_info_3d['collector'] += frame_hist
        hist_mon_info_3d['collector_std'] += np.square(frame_hist)

    lastname = 'Crd'
    np.save(
        save_to + sim_name + '-XyzHist' + lastname + '.npy',
        hist_crd_info_3d['collector']
    )
    np.save(
        save_to + sim_name + '-XyzHistStd' + lastname + '.npy',
        hist_crd_info_3d['collector_std']
    )
    np.save(
        save_to + sim_name + '-XyzEdges' + lastname + '.npy',
        hist_crd_info_3d['bin_edges']
    )
    lastname = 'Mon'
    np.save(
        save_to + sim_name + '-XyzHist' + lastname + '.npy',
        hist_mon_info_3d['bin_edges']
    )
    np.save(
        save_to + sim_name + '-XyzHistStd' + lastname + '.npy',
        hist_mon_info_3d['collector_std']
    )
    np.save(
        save_to + sim_name + '-XyzEdges' + lastname + '.npy',
        hist_mon_info_3d['collector_std']
    )
    print('done.')

['/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000adump5000ens7.j01.all.lammpstrj', '/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000adump5000ens7.j02.all.lammpstrj', '/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000adump5000ens7.j03.all.lammpstrj', '/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000adump5000ens7.j05.all.lammpstrj']
['/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000adump5000ens7.j02.all.lammpstrj', '/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000adump5000ens7.j01.all.lammpstrj', '/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000adump5000ens7.j05.all.lammpstrj', '/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc3203dt0.005bdump1000

In [ ]:
# 4 mins for ~4000 particles with one all trj
# macbookmini
parent = "/Users/amirhsi_mini/research_data/trjs"
# macbookpro
#parent = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(parent + '/N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
print(all_trjs)
all_topo = glob(parent + '/N*' + group + '*')
print(all_topo)
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs[:1]:
    trj_info = SumRule(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        sum_rule_all_histdd(
            all_topo, all_trj, geometry, lineage, save_to=save_to
        )
    else:
        sum_rule_all_histdd(
            all_topo,
            all_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )

### Trans-Foci project

#### whole bug trjs

In [ ]:
# analyzing bug files.
geometry = 'biaxial'
group = 'bug'
linage = 'whole'
save_to = './'
#macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
bug_pairs = glob(macbookpro_path + '/eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug(
        bug_topo,
        bug_trj,
        geometry,
        lineage,
        save_to = save_to
    )

#### segment all trjs

In [ ]:
# 1 min and 45 s for ~30000 particles with one all trj
#macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
macmini_path = '/Users/amirhsi_mini/research_data/test'
geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macmini_path + '/eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macmini_path + '/eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFoci(
        all_trj,
        geometry=geometry,
        group=group,
        lineage=topo_lineage
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all(
            all_topo, all_trj, geometry, lineage, save_to=save_to
        )
    else:

        prober.trans_foci_all(
            all_topo,
            all_trj,
            geometry,
            lineage,
            save_to=save_to,
            continuous=True
        )